In [ ]:
import logging
import json
import codecs
import pandas as pd
import requests
from bson import json_util
from github import Github
import xlsxwriter
import numpy as np
from itertools import cycle
import git
from git import Repo
from github import Github
from git import Repo, Commit
import os
import sys
from pymongo import MongoClient
import io
import nltk
import re
from collections import defaultdict
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import SnowballStemmer
import mysql.connector
import csv
import operator
import matplotlib.pyplot as plt
import re

## 1. BD CONNECTION

In [ ]:
mydb = mysql.connector.connect(user='obfuscate', password='obfuscate', host='obfuscate', database='obfuscate')

In [ ]:
mycursor = mydb.cursor(buffered=True)

## 2. Getting list of IDS

In [ ]:
def get_ids_to_consider():
    list_ids = set()
    i=1
    with open('ids_to_consider.csv', newline='') as csvfile:
        reader = csv.reader(csvfile)
        for row in reader:
            ##print(str(i),row[0])
            if row[0] in list_ids:
                print(row[0])
            list_ids.add(row[0])
            
            i=i+1
    return list_ids

In [ ]:
list_ids = get_ids_to_consider()

In [ ]:
print(str(len(list_ids)))

In [ ]:
def validate_language(set_of_ids):
    java = 0
    kotlin = 0
    query = "SELECT type FROM entity WHERE id="
    for element in set_of_ids:    
        mycursor.execute(query+str(element))
        for lang in mycursor:
            language = lang[0]
            if language == 'java':
                java=java+1
            else:
                kotlin=kotlin+1
    return (java,kotlin)

In [ ]:
print(validate_language(list_ids))

In [ ]:
def validate_same_vuln_tag(set_of_ids):
    list_ids_good = set()
    list_ids_wrong = set()
    
    query = "SELECT vulnerability_tag FROM tag WHERE entity_id="
    for element in set_of_ids:    
        mycursor.execute(query+str(element))
        
        unique_tag = set()
        
        for tagged_vuln in mycursor:
            tagged_vulnerability = tagged_vuln[0]
            unique_tag.add(tagged_vulnerability)
            #if('unclear' == tagged_vulnerability):
                #print(element)
        if(len(unique_tag)==1):
            list_ids_good.add(element)
        else:
            list_ids_wrong.add(element)

    return (list_ids_good,list_ids_wrong)
    

In [ ]:
validation_same_v_tag = validate_same_vuln_tag(list_ids)

In [ ]:
print(str(len(validation_same_v_tag[0])))

In [ ]:
print(validation_same_v_tag[1])

## 3. Get list of elements for each language

In [ ]:
def validate_same_vuln_tag(set_of_ids):
    list_unique_vuln_tags_kotlin = dict()
    list_unique_vuln_tags_java = dict()
    
    query = "SELECT tag.entity_id, entity.type, tag.vulnerability_tag FROM tag INNER JOIN entity ON entity.id=tag.entity_id and entity_id="
    for element in set_of_ids:    
        mycursor.execute(query+str(element)+" limit 1")
        
        for tagged_vuln in mycursor:
            
            vulnerability = tagged_vuln[2]
            language = tagged_vuln[1]
            
            if language == "java":
                if vulnerability in list_unique_vuln_tags_java:
                    list_unique_vuln_tags_java[vulnerability] += 1
                else:
                    list_unique_vuln_tags_java[vulnerability] = 1
            elif language == "kt":
                if vulnerability in list_unique_vuln_tags_kotlin:
                    list_unique_vuln_tags_kotlin[vulnerability] += 1
                else:
                    list_unique_vuln_tags_kotlin[vulnerability] = 1
    
    return (list_unique_vuln_tags_kotlin, list_unique_vuln_tags_java)
            

In [ ]:
dictionaries = validate_same_vuln_tag(list_ids)

## 3.1 KOTLIN

In [ ]:
dictionary_kotlin = dictionaries[0]

In [ ]:
print(str(len(dictionary_kotlin)))

In [ ]:
sorted_d_kotlin = dict(sorted(dictionary_kotlin.items(), key=operator.itemgetter(1),reverse=True))

In [ ]:
print(sorted_d_kotlin)

In [ ]:
plt.bar(range(len(sorted_d_kotlin)), list(sorted_d_kotlin.values()), align='center')
#plt.xticks(range(len(sorted_d_kotlin)), list(sorted_d_kotlin.keys()))


plt.show()

In [ ]:
sum(sorted_d_kotlin.values())

In [ ]:
for key in sorted_d_kotlin.keys():
    print(key)

In [ ]:
for k, v in sorted_d_kotlin.items():
    print(k, v)

## 3.2 JAVA

In [ ]:
dictionary_java = dictionaries[1]

In [ ]:
print(str(len(dictionary_java)))

In [ ]:
sorted_d_java = dict(sorted(dictionary_java.items(), key=operator.itemgetter(1),reverse=True))

In [ ]:
print(sorted_d_java)

In [ ]:
plt.bar(range(len(sorted_d_java)), list(sorted_d_java.values()), align='center')
#plt.xticks(range(len(sorted_d_java)), list(sorted_d_java.keys()))


plt.show()

In [ ]:
sum(sorted_d_java.values())

In [ ]:
for key in sorted_d_java.keys():
    print(key)

In [ ]:
for k, v in sorted_d_java.items():
    print(k, v)

## 4. Concatening Data views

In [ ]:
first_view = pd.read_csv("CWE_list/699.csv", sep = ",", index_col=False)
first_view.head(1)

In [ ]:
first_view.shape

In [ ]:
second_view = pd.read_csv("CWE_list/1000.csv", sep = ",",  index_col=False)
second_view.head(1)

In [ ]:
second_view.shape

In [ ]:
third_view = pd.read_csv("CWE_list/1194.csv", sep = ",",  index_col=False)
third_view.head(1)

In [ ]:
third_view.shape

In [ ]:
all_views  = pd.concat([first_view, second_view, third_view])
all_views.head(1)

In [ ]:
all_views.shape

In [ ]:
all_views = all_views.drop_duplicates(subset=['CWE-ID'])

In [ ]:
all_views.shape

In [ ]:
short_version_all_views  = all_views[['CWE-ID','Name','Related Weaknesses']]

In [ ]:
short_version_all_views.head(1)

In [ ]:
short_version_all_views.shape

## 5. REGEX for CWE

In [ ]:
def get_list_of_childs(dataframe):
    
    dictionary = dict()
    dictionary.update( {'0' : set()} )
    
    for index, row in dataframe.iterrows():
        current_cwe = str(row['CWE-ID'])
        dictionary.update( {current_cwe : set()} )
    
    for index, row in dataframe.iterrows():
        try:
            current_cwe = str(row['CWE-ID'])
            current_parents = row['Related Weaknesses']
            
            if isinstance(current_parents, str):
            
                list_parents = [match.group().split(":")[2] for match in re.finditer("ChildOf:CWE ID:[0-9]+", row['Related Weaknesses'])]      
                if list_parents:
                    for element in list_parents:
                        dictionary[element].add(current_cwe)
                else:
                    dictionary['0'].add(current_cwe)
            else:
                list_parents = [match.group().split(":")[2] for match in re.finditer("ChildOf:CWE ID:[0-9]+", str(row['Related Weaknesses']))]      
                if list_parents:
                    for element in list_parents:
                        dictionary[element].add(current_cwe)
                else:
                    dictionary['0'].add(current_cwe)
               
        except Exception as inst:
            print(row)
            continue
    
    return dictionary
    

In [ ]:
graph = get_list_of_childs(short_version_all_views)

In [ ]:
def get_info(set_of_ids):
    for element in set_of_ids:    
        mycursor.execute("SELECT id, text, type, repository,vulnerability_tag FROM  ( select * from entity WHERE id="+"'"+str(element)+"'"+") as partial INNER JOIN  tag ON partial.id=tag.entity_id LIMIT 1")
        for lang in mycursor:
            print(lang)
            ## I added special characters at the beggining and at the end in order to use 'replace' in VSCode and remove the starting '(' and ending ')'

In [ ]:
get_info(list_ids)